In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import json
import ast
import numpy as np
import pickle
import collections
from collections import Counter
from fastai import *
from fastai.text import *

In [4]:
data = []
with open('data/qa_Beauty.json') as f:
    for line in f:
        data.append(ast.literal_eval(line))
        
qa_beauty = pd.DataFrame(data)
qa_beauty.head()

,answer,answerTime,answerType,asin,question,questionType,unixTime
0,"yes it comes with adjustable dividers, you can...",10 days ago,Y,602260074X,can you fit make up brushes in the trays,yes/no,1.425370e+09
1,"yes,all the provided dividers are adjustable","Mar 3, 2015",?,602260074X,Can you move all the dividers?,yes/no,1.425370e+09
2,Yes,"Dec 30, 2014",Y,602260074X,is the surface in side the smooth?,yes/no,1.419926e+09
3,"Hi there, Not too deep. Maybe like an inch dee...","Jul 13, 2014",NaN,602260074X,How deep do the extending trays measure?,open-ended,1.405235e+09
4,No. We just tried it and it won't.,"May 21, 2014",?,602260074X,Can bottles of nail polish stand upright in th...,yes/no,1.400656e+09


In [5]:
qa_beauty.shape

(42422, 7)

In [6]:
qa_beauty['asin'].unique()

array(['602260074X', '7800558258', '9788071813', ..., 'B00KYWMYEE',
       'B00L5DI5K8', 'B00L5JHZJO'], dtype=object)

In [7]:
data = []
with open('data/reviews_Beauty_5.json') as f:
    for line in f:
        data.append(ast.literal_eval(line))
        
reviewsBeauty = pd.DataFrame(data)
reviewsBeauty.head()

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,7806397051,"[3, 4]",1.0,Very oily and creamy. Not at all what I expect...,"01 30, 2014",A1YJEY40YUW4SE,Andrea,Don't waste your money,1391040000
1,7806397051,"[1, 1]",3.0,This palette was a decent price and I was look...,"04 18, 2014",A60XNB876KYML,Jessica H.,OK Palette!,1397779200
2,7806397051,"[0, 1]",4.0,The texture of this concealer pallet is fantas...,"09 6, 2013",A3G6XNM240RMWA,Karen,great quality,1378425600
3,7806397051,"[2, 2]",2.0,I really can't tell what exactly this thing is...,"12 8, 2013",A1PQFP6SAJ6D80,Norah,Do not work on my face,1386460800
4,7806397051,"[0, 0]",3.0,"It was a little smaller than I expected, but t...","10 19, 2013",A38FVHZTNQ271F,Nova Amor,It's okay.,1382140800


In [5]:
# import nltk
# nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/pranav_shukla/nltk_data...


True

In [8]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
vader_analyzer = SentimentIntensityAnalyzer()

/home/pranav_shukla/anaconda3/lib/python3.6/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [9]:
def find_sentiment(row):
    sent_dict = vader_analyzer.polarity_scores(row['summary'])
    if (sent_dict['neg']+sent_dict['neu']) > sent_dict['pos']:
        return 0
    else:
        return 1

reviewsBeauty['pos_sentiment'] = reviewsBeauty.apply(find_sentiment, axis=1)

In [29]:
def helpfulness_score(row):
    if row['helpful'][1] !=0:
        return row['helpful'][0]/row['helpful'][1]
    else:
        return 0

reviewsBeauty['helpfulness_score'] = reviewsBeauty.apply(helpfulness_score, axis=1)

In [30]:
reviewsBeauty.head()

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime,pos_sentiment,helpfulness_score
0,7806397051,"[3, 4]",1.0,Very oily and creamy. Not at all what I expect...,"01 30, 2014",A1YJEY40YUW4SE,Andrea,Don't waste your money,1391040000,0,0.75
1,7806397051,"[1, 1]",3.0,This palette was a decent price and I was look...,"04 18, 2014",A60XNB876KYML,Jessica H.,OK Palette!,1397779200,1,1.00
2,7806397051,"[0, 1]",4.0,The texture of this concealer pallet is fantas...,"09 6, 2013",A3G6XNM240RMWA,Karen,great quality,1378425600,1,0.00
3,7806397051,"[2, 2]",2.0,I really can't tell what exactly this thing is...,"12 8, 2013",A1PQFP6SAJ6D80,Norah,Do not work on my face,1386460800,0,1.00
4,7806397051,"[0, 0]",3.0,"It was a little smaller than I expected, but t...","10 19, 2013",A38FVHZTNQ271F,Nova Amor,It's okay.,1382140800,1,0.00


In [11]:
qa_beauty.shape, reviewsBeauty.shape

((42422, 7), (198502, 10))

In [12]:
qa_beauty['question'].isnull().values.any(), qa_beauty['answer'].isnull().values.any()

(False, False)

In [10]:
# df = pd.merge(qa_beauty, reviewsBeauty, how='inner', on='asin')
# df.asin.unique()

In [13]:
ques = list(qa_beauty['question'])
ans = list(qa_beauty['answer'])

In [14]:
len(ques), len(ans)

(42422, 42422)

In [16]:
ques_tok = Tokenizer().process_all(ques)
ans_tok = Tokenizer().process_all(ans)

In [ ]:
ques[0], ques_tok[0]

In [ ]:
ans[0], ans_tok[0]

In [ ]:
np.percentile([len(o) for o in ques_tok], 90), np.percentile([len(o) for o in ans_tok], 90)

In [ ]:
keep_ques = np.array([len(o)<33 for o in ques_tok])
keep_ans = np.array([len(o)<83 for o in ans_tok])

In [ ]:
keep = keep_ques*keep_ans

In [ ]:
ques_tok = np.array(ques_tok)[keep]
ans_tok = np.array(ans_tok)[keep]

In [ ]:
len(ques_tok), len(ans_tok)

In [ ]:
with open('data/ques_tok.pickle', 'wb') as file:
    pickle.dump(ques_tok, file)
with open('data/ans_tok.pickle', 'wb') as file:
    pickle.dump(ans_tok, file)

In [ ]:
with open('data/ques_tok.pickle', 'rb') as file:
    ques_tok = pickle.load(file)
with open('data/ans_tok.pickle', 'rb') as file:
    ans_tok = pickle.load(file)

In [ ]:
def toks2ids(tok):
    freq = Counter(p for o in tok for p in o)
    itos = [o for o,c in freq.most_common(40000)]
    itos.insert(0, '_bos_')
    itos.insert(1, '_pad_')
    itos.insert(2, '_eos_')
    itos.insert(3, '_unk')
    stoi = collections.defaultdict(lambda: 3, {v:k for k,v in enumerate(itos)})
    ids = np.array([([stoi[o] for o in p] + [2]) for p in tok])
    return ids,itos,stoi

In [17]:
ques_ids,ques_itos,ques_stoi = toks2ids(ques_tok)
ans_ids,ans_itos,ans_stoi = toks2ids(ans_tok)

In [18]:
[ans_itos[o] for o in ans_ids[0]], len(ques_itos), len(ans_itos)

## Word Vectors

In [19]:
import fastText as ft
from pyfasttext import FastText

To use the fastText library, you'll need to download fasttext word vectors for your language (download the 'bin plus text' ones).

In [26]:
en_vecs = FastText('data/wiki.en.bin')

In [27]:
en_vecs

In [28]:
enc_vecd = {w:en_vecs[w] for w in en_vecs.words}

In [29]:
dim_en_vec = len(enc_vecd[','])
dim_en_vec

300

In [30]:
en_vecs = np.stack(list(enc_vecd.values()))
en_vecs.mean(),en_vecs.std()

(0.0075652334, 0.29283327)

## Model Data

In [31]:
queslen_99 = int(np.percentile([len(o) for o in ques_ids], 99))
anslen_97 = int(np.percentile([len(o) for o in ans_ids], 97))
queslen_99,anslen_97

(33, 73)

In [32]:
ques_ids_tr = np.array([o[:queslen_99] for o in ques_ids])
ans_ids_tr = np.array([o[:anslen_97] for o in ans_ids])

In [33]:
class Seq2SeqDataset(Dataset):
    def __init__(self, x, y): self.x,self.y = x,y
    def __getitem__(self, idx): return A(self.x[idx], self.y[idx])
    def __len__(self): return len(self.x)

In [34]:
np.random.seed(42)
trn_keep = np.random.rand(len(ques_ids_tr))>0.1
ques_trn,ans_trn = ques_ids_tr[trn_keep],ans_ids_tr[trn_keep]
ques_val,ans_val = ques_ids_tr[~trn_keep],ans_ids_tr[~trn_keep]
len(ques_trn),len(ques_val)

(31382, 3507)

In [35]:
trn_ds = Seq2SeqDataset(ques_trn,ans_trn)
val_ds = Seq2SeqDataset(ques_val,ans_val)

In [36]:
bs=125

In [37]:
trn_samp = SortishSampler(ques_trn, key=lambda x: len(ques_trn[x]), bs=bs)
val_samp = SortSampler(ques_val, key=lambda x: len(ques_val[x]))

In [54]:
PATH = 'models/'
trn_dl = DataLoader(trn_ds, bs, transpose=True, transpose_y=True, num_workers=1, pad_idx=1, pre_pad=False, sampler=trn_samp)
val_dl = DataLoader(val_ds, int(bs*1.6), transpose=True, transpose_y=True, num_workers=1, pad_idx=1, pre_pad=False, 
                    sampler=val_samp)
md = ModelData(PATH, trn_dl, val_dl)

In [21]:
it = iter(trn_dl)
its = [next(it) for i in range(5)]
[(len(x),len(y)) for x,y in its]

[(33, 29), (21, 7), (21, 8), (33, 13), (33, 21)]

## Attention Model

In [ ]:
def rand_t(*sz): return torch.randn(sz)/math.sqrt(sz[0])
def rand_p(*sz): return nn.Parameter(rand_t(*sz))

In [ ]:
class Seq2SeqAttnRNN(nn.Module):
    def __init__(self, vecs_enc, itos_enc, em_sz_enc, vecs_dec, itos_dec, em_sz_dec, nh, out_sl, nl=2):
        super().__init__()
        self.emb_enc = create_emb(vecs_enc, itos_enc, em_sz_enc)
        self.nl,self.nh,self.out_sl = nl,nh,out_sl
        self.gru_enc = nn.GRU(em_sz_enc, nh, num_layers=nl, dropout=0.25)
        self.out_enc = nn.Linear(nh, em_sz_dec, bias=False)
        self.emb_dec = create_emb(vecs_dec, itos_dec, em_sz_dec)
        self.gru_dec = nn.GRU(em_sz_dec, em_sz_dec, num_layers=nl, dropout=0.1)
        self.emb_enc_drop = nn.Dropout(0.15)
        self.out_drop = nn.Dropout(0.35)
        self.out = nn.Linear(em_sz_dec, len(itos_dec))
        self.out.weight.data = self.emb_dec.weight.data

        self.W1 = rand_p(nh, em_sz_dec)
        self.l2 = nn.Linear(em_sz_dec, em_sz_dec)
        self.l3 = nn.Linear(em_sz_dec+nh, em_sz_dec)
        self.V = rand_p(em_sz_dec)

    def forward(self, inp, y=None, ret_attn=False):
        sl,bs = inp.size()
        h = self.initHidden(bs)
        emb = self.emb_enc_drop(self.emb_enc(inp))
        enc_out, h = self.gru_enc(emb, h)
        h = self.out_enc(h)

        dec_inp = V(torch.zeros(bs).long())
        res,attns = [],[]
        w1e = enc_out @ self.W1
        for i in range(self.out_sl):
            w2h = self.l2(h[-1])
            u = F.tanh(w1e + w2h)
            a = F.softmax(u @ self.V, 0)
            attns.append(a)
            Xa = (a.unsqueeze(2) * enc_out).sum(0)
            emb = self.emb_dec(dec_inp)
            wgt_enc = self.l3(torch.cat([emb, Xa], 1))
            
            outp, h = self.gru_dec(wgt_enc.unsqueeze(0), h)
            outp = self.out(self.out_drop(outp[0]))
            res.append(outp)
            dec_inp = V(outp.data.max(1)[1])
            if (dec_inp==1).all(): break
            if (y is not None) and (random.random()<self.pr_force):
                if i>=len(y): break
                dec_inp = y[i]

        res = torch.stack(res)
        if ret_attn: res = res,torch.stack(attns)
        return res

    def initHidden(self, bs): return V(torch.zeros(self.nl, bs, self.nh))

In [ ]:
rnn = Seq2SeqAttnRNN(ans_vecd, ans_itos, dim_ans_vec, ques_vecd, ques_itos, dim_ques_vec, nh, queslen_99)
learn = RNN_Learner(md, SingleModel(to_gpu(rnn)), opt_fn=opt_fn)
learn.crit = seq2seq_loss

In [ ]:
lr=2e-3

In [ ]:
learn.fit(lr, 1, cycle_len=15, use_clr=(20,10), stepper=Seq2SeqStepper)

### Test

### Take the best of reviews based on helpfulness of a review

In [ ]:
# Assume that we are given the question and asin
x,y = next(iter(val_dl))
probs,attns = learn.model(V(x),ret_attn=True)
preds = to_np(probs.max(2)[1])
sample_ques = ' '.join([fr_itos[o] for o in x[:,23] if o != 1])
predicted_ans = ' '.join([en_itos[o] for o in preds[:,23] if o!=1])

sent_dict = vader_analyzer.polarity_scores(predicted_ans)
predicted_ans_sent = ((sent_dict['neg']+sent_dict['neu']) > sent_dict['pos']) ? 0:1

In [46]:
sample_asin = '7806397051'
predicted_ans_sent = 0 # negative sentiments

In [49]:
top_3_review_compl_sentiment = reviewsBeauty[(reviewsBeauty.asin == sample_asin) & (reviewsBeauty.pos_sentiment == 1)].nlargest(3, 'helpfulness_score')['reviewText']

In [50]:
top_3_review_compl_sentiment # predicted complimentory reviews

1    This palette was a decent price and I was look...
5    I was very happy to get this palette, now I wi...
2    The texture of this concealer pallet is fantas...
Name: reviewText, dtype: object